# MultiNomial Regression (Softmax Clf)

* [강의자료](https://www.youtube.com/watch?v=B3gtAi-wlG8&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=9)
* [코드자료](https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-06_1_softmax_classification.ipynb)
* [코드자료](https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-06_2_fancy_softmax_classification.ipynb)

## Module

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(42)

In [3]:
# data
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
x_train.shape, y_train.shape

(torch.Size([8, 4]), torch.Size([8]))

In [4]:
# weights
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [5]:
# optimizer
optimizer = optim.SGD([W,b], lr=0.1)

In [6]:
nb_epochs = 1000
for epoch in range(1, nb_epochs + 1):
    
    # hypothesis - softmax
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    # y_one_hot
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    
    # cost function
    cost = -(y_one_hot * torch.log(hypothesis)).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch  100/1000 Cost: 0.714269
Epoch  200/1000 Cost: 0.638430
Epoch  300/1000 Cost: 0.591186
Epoch  400/1000 Cost: 0.554105
Epoch  500/1000 Cost: 0.522028
Epoch  600/1000 Cost: 0.492792
Epoch  700/1000 Cost: 0.465232
Epoch  800/1000 Cost: 0.438588
Epoch  900/1000 Cost: 0.412274
Epoch 1000/1000 Cost: 0.385798


In [7]:
# scratch
hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
cost = -(y_one_hot * torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

# same 1
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
cost = -(y_one_hot * F.log_softmax(x_train.matmul(W) + b, dim=1)).sum(dim=1).mean()
print(cost)

# same 2
print(F.nll_loss(F.log_softmax(x_train.matmul(W) + b, dim=1), y_train))

# same 3
print(F.cross_entropy(x_train.matmul(W) + b, y_train))

tensor(0.4000, grad_fn=<NegBackward>)
tensor(0.4000, grad_fn=<NegBackward>)
tensor(0.4000, grad_fn=<NllLossBackward>)
tensor(0.4000, grad_fn=<NllLossBackward>)


## using pytorch method

In [8]:
class SoftmaxClassifier(nn.Module):
    def __init__(self, input_shape, output_shape):
        super().__init__()
        self.linear = nn.Linear(input_shape, output_shape)
    
    def forward(self, x):
        return self.linear(x)

In [9]:
# constants
input_shape = x_train.shape[-1]
output_shape = y_train.unique().__len__()
learning_rate = 0.1

In [10]:
# model and optimizer
model = SoftmaxClassifier(input_shape, output_shape)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
nb_epochs = 1000
for epoch in range(1, nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch  100/1000 Cost: 0.670299
Epoch  200/1000 Cost: 0.581678
Epoch  300/1000 Cost: 0.525373
Epoch  400/1000 Cost: 0.479959
Epoch  500/1000 Cost: 0.439910
Epoch  600/1000 Cost: 0.402833
Epoch  700/1000 Cost: 0.367230
Epoch  800/1000 Cost: 0.331915
Epoch  900/1000 Cost: 0.295977
Epoch 1000/1000 Cost: 0.260169
